In [27]:
import datetime
import pytz
import warnings

%run ./common.ipynb

warnings.filterwarnings("ignore")

In [28]:
def format_timedelta(td):
    """Formats a timedelta object to 'day.hours:min:sec' format."""

    seconds = td.total_seconds()
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    return f"{int(days)}.{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

In [29]:
def calculateCycleTime_NewWorkFlow(_status_change,current_status,key,proj):
    cycleTime = {}
    if (current_status != 'Done'):
        # Get the current time in UTC
        now_utc = datetime.datetime.now(pytz.utc)
        # Convert to Eastern Time
        eastern = pytz.timezone('US/Eastern')
        now_est = now_utc.astimezone(eastern)
        new_node = {
                    'proj':proj,
                    'key':key,
                    'created_at': now_est.strftime('%Y-%m-%dT%H:%M:%S.%f%z'),
                    # 'field': 'status',
                    'from': current_status,
                    'to' : 'Active'
                    }
        _status_change.loc[len(_status_change)] = new_node     
    
    _status_change = _status_change.reset_index(drop=True)
    _status_change['created_at'] = pd.to_datetime(_status_change['created_at'], format='%Y-%m-%dT%H:%M:%S.%f%z')
    _status_change['time_diff'] = _status_change['created_at'].diff()
    _status_change['proj']=proj
    _status_change = _status_change.reset_index(drop=True)
    _status_change.to_csv('./output/transistion_history.csv', mode='a', header=False, index=False)
    cycleTime['proj'] = proj
    cycleTime['key'] = _status_change['key'][0]
    cycleTime['leadTime'] = format_timedelta(_status_change['time_diff'].sum())
    cycleTime['development'] = format_timedelta(_status_change[_status_change['to'].isin( ['In Progress','Code Review','Testing','Active'])]['time_diff'].sum())
    cycleTime['QA'] = format_timedelta(_status_change[_status_change['from'].isin( ['Testing']) ] [_status_change['to'].isin( ['Deploy', 'Active'])]['time_diff'].sum())
    cycleTime['deployment'] = format_timedelta(_status_change[_status_change['from'].isin( ['Deploy']) ]['time_diff'].sum())

    # display (_status_change)
    # display(cycleTime)
    return cycleTime

In [30]:
def getChangeLog(searchQry_DS,proj):
    changelog_Rec =[ ]
    cycleTimeDf = pd.DataFrame(columns=['key','leadTime','development','QA','deployment','proj'])

    for searchResult in searchQry_DS:
        changelog = searchResult['changelog']
        for history in changelog['histories']:
            history_id = history['id']
            created_at = history.get('created',None)
            for item in history['items']:
                changelog_Rec.append({
                    'proj':proj,
                    'key': searchResult['key'],
                    'created_at': created_at,
                    'field': item['field'],
                    'from': item['fromString'],
                    'to': item['toString'],
                })
    if changelog_Rec :
        changelog_Rec_df = pd.DataFrame(changelog_Rec)

        status_change = changelog_Rec_df[changelog_Rec_df['field']=='status']

        status_change = status_change.reset_index(drop=True)
        for searchResult in searchQry_DS:
            result = status_change[status_change['key'] == searchResult['key']]
            cycleTimeDf = cycleTimeDf.append(calculateCycleTime_NewWorkFlow(result,searchResult['fields']['status']['name'],searchResult['key'],proj), ignore_index=True)

    return cycleTimeDf

In [31]:
def cycleTimeForAProject(proj):
    searchQry='?jql= project in ("%s") and type  in (Story) and updated >= startOfMonth(-3)&fields=key,status&expand=changelog&maxResults=5000'%"".join(proj)
    searchQry_DS = getDataSet(searchQry)
    _cycleTimeDataFrame = pd.DataFrame()
    if searchQry_DS:
       _cycleTimeDataFrame = getChangeLog(searchQry_DS,proj)
    # display(cycleTimeDf)
    return _cycleTimeDataFrame
